# M2M PLIMS Data Access Testing

#### Intro
Starting with the first deployment of the Coastal Pioneer Mid-Atlantic Bight (MAB) Array, OOI collected data from a plankton imaging instrument (McLane Research Labs Imaging Flow Cytobot) deployed at 7 meters depth on the near-surface instrument frame. The introduction of a new instrument to the physical OOI infrastructure required new parsing code and metadata to ingest the data into the OOI Cyberinfrastructure. Once ingested, the data will be available for machine-to-machine (M2M) download from OOI's THREDDS server and then pushed to Data Explorer on a near-real-time basis.

#### Purpose
Telemetered data from the first PLIMS deployment on Central Surface Mooring has been ingested into the Dev2 development platform where the relevant parsers are being tested. After the parameters and relevant metadata were decided upon, test NetCDFs were created for review to confirm that the parsers could create a valid and accurate dataset. This data pathway from ingested data to datasets asynchronously compiled on demand now must be tested from the perspective of a data user.

#### Objectives:

* Test Output: Request HDR science and engineering datasets with different request parameters.
* Dataset Review: Look for incorrect data values, types or metadata to provide feedback to CI. 

#### Supporting tools
If you want to run this notebook as-is, you will need to clone A. Reed's [ooinet repo](https://github.com/reedan88/OOINet) to your local machine and install it as a local dev repo (which adds it to your base path). You'll also need the [ooi_data_explorations repo](https://github.com/oceanobservatories/ooi-data-explorations). Lastly, you'll want to install the ```ioos_qc``` [python package](https://github.com/ioos/ioos_qc). 


In [2]:
# Import libraries
import os
import re
import gc
import io
import ast
import netrc
import pandas as pd
import numpy as np
import xarray as xr
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import glob

# Import dask
import dask
from dask.diagnostics import ProgressBar

# Import pyplot and show plots inline
import matplotlib.pyplot as plt
%matplotlib inline 

Import the M2M tool

In [3]:
from ooinet import M2M
from ooinet.Instrument.common import process_file

Import the relevant ooi_data_explorations tools

In [4]:
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations.common import get_vocabulary

---
## Request and load the data
Sub in ooinet-dev0-west.intra.oceanobservatories.org into the avaialbe API urls


In [5]:
for key in M2M.URLS:
    url = M2M.URLS.get(key)
    if "opendap" in url:
        M2M.URLS[key] = re.sub("opendap", "opendap-dev0-west.intra", url)
    else:
        M2M.URLS[key] = re.sub("ooinet","ooinet-dev0-west.intra", url)

Sub in dev0 OOINet credentials from netrc file

In [6]:
try:
    nrc = netrc.netrc()
    M2M.AUTH = nrc.authenticators('ooinet-dev0-west.intra.oceanobservatories.org')
    M2M.login, M2M.password = M2M.AUTH[0], M2M.AUTH[2]
    if M2M.AUTH is None:
        raise RuntimeError(
            'No entry found for machine ``ooinet-dev0-west.intra.oceanobservatories.org`` in the .netrc file')
except FileNotFoundError as e:
    raise OSError(e, os.strerror(e.errno), os.path.expanduser('~'))

Search the Dev1 server for available datasets

In [7]:
M2M.URLS

{'data': 'https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12576/sensor/inv',
 'anno': 'https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12580/anno/find',
 'vocab': 'https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12586/vocab/inv',
 'asset': 'https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12587',
 'deploy': 'https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12587/events/deployment/inv',
 'preload': 'https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12575/parameter',
 'cal': 'https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12587/asset/cal',
 'fileServer': 'https://opendap-dev0-west.intra.oceanobservatories.org/thredds/fileServer/',
 'dodsC': 'https://opendap-dev0-west.intra.oceanobservatories.org/thredds/dodsC/',
 'goldCopy': 'https://thredds.dataexplorer.oceanobservatories.org/thredds/catalog/ooigoldcopy/public/',
 'goldCopy_fileServer': 'https://thredds.dataexplorer.oceanobservatories.org/thre

In [8]:
ds_df = M2M.search_datasets(array="CP10CNSM")
ds_df.reset_index(inplace=True)
# ds_df

Searching https://ooinet-dev0-west.intra.oceanobservatories.org/api/m2m/12576/sensor/inv/CP10CNSM


Find the available datastreams for a given **refdes**

In [ ]:
# url = ds_df["url"][0]
# url = "/".join((url, 'telemetered', 'plims_a_hdr_instrument'))
# # Query the preload data
# preload_data = M2M.get_api(url)
# preload_data

NOT FOUND


In [9]:
datastreams = M2M.get_datastreams("CP10CNSM-RID27-07-PLIMSA000")
datastreams

,refdes,method,stream
0,CP10CNSM-RID27-07-PLIMSA000,recovered_host,plims_a_adc_instrument
1,CP10CNSM-RID27-07-PLIMSA000,recovered_host,plims_a_hdr_engineering
2,CP10CNSM-RID27-07-PLIMSA000,recovered_host,plims_a_hdr_instrument
3,CP10CNSM-RID27-07-PLIMSA000,telemetered,plims_a_adc_instrument
4,CP10CNSM-RID27-07-PLIMSA000,telemetered,plims_a_hdr_engineering
5,CP10CNSM-RID27-07-PLIMSA000,telemetered,plims_a_hdr_instrument


In [10]:
# Define a generic preprocessing routine. Do NOT use any of the ooi_data_explorations "process_instrument" methods. We want to be comparing "apples-to-apples" 
def preprocess(ds):
    ds = process_file(ds)
    return ds

In [11]:
# Define function to change primary dimension to the sample start timestamp
def swap_timestamps(ds):
    """
    Swaps the timestamps from the host to the instrument timestamp
    for the CTDBPs
    """
    if "internal_timestamp" in ds.variables:
        # Calculate the timestamp
        inst_time = ds.internal_timestamp.to_pandas()
        attrs = ds.internal_timestamp.attrs
        # Convert the time
        inst_time = inst_time.apply(lambda x: np.datetime64(int(x), 's'))
        # Create a DataArary
        da = xr.DataArray(inst_time, attrs=attrs)
        ds['internal_timestamp'] = da
    ds = ds.set_coords(["internal_timestamp"])
    ds = ds.swap_dims({"time":"internal_timestamp"})
    ds = ds.reset_coords("time")
    ds = ds.rename_vars({"time":"host_time"})
    ds["host_time"].attrs = {
        "long_name": "DCL Timestamp",
        "comment": ("The timestamp that the instrument data as recorded by the mooring data "
                    "concentration logger (DCL)")
    }
    ds = ds.rename({"internal_timestamp":"time"})
    return ds

#### Telemetered HDR engineering data stream

In [12]:
# Setup parameters needed to request data
refdes = datastreams.refdes[4]
method = datastreams.method[4]
stream = datastreams.stream[4]

In [13]:
# Import vocabulary
site, node, sensor = refdes.split('-', 2)
vocab = get_vocabulary(site, node, sensor)

In [14]:
# Use the gold copy THREDDs datasets
thredds_url = M2M.get_thredds_url(refdes, method, stream, beginDT="2024-05-01T00:00:00.0", endDT="2024-05-15T00:00:00.0")
# Get the THREDDs catalog
thredds_catalog = M2M.get_thredds_catalog(thredds_url)
thredds_catalog = [x for x in thredds_catalog if "blank" not in x]

# Clean the THREDDs catalog
sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream)

# Generate the urls to access and load the data
sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["dodsC"], file) for file in sensor_files]



Waiting for request to process
Waiting for request to process
Waiting for request to process
Waiting for request to process
Waiting for request to process
Waiting for request to process


In [15]:
# Load the data
with ProgressBar():
    data_hdreng = xr.open_mfdataset(sensor_files, parallel=True, preprocess=preprocess)

[########################################] | 100% Completed | 1.33 ss


In [16]:
# Check dataset content
data_hdreng

<xarray.Dataset> Size: 21kB
Dimensions:               (time: 83)
Coordinates:
  * time                  (time) datetime64[ns] 664B 2024-05-01T00:25:09 ... ...
Data variables: (12/45)
    pump1_state           (time) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    refill_debubble       (time) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    prime_sample          (time) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    deployment            (time) int32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    alt_interval          (time) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    internal_temperature  (time) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    ...                    ...
    alt_pmta_threshold    (time) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    focus_motor_step_lg   (time) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    syringes_autorun      (time) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    bleach_rinse_ct       (time) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    auto_start            (time) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_volume_skip    (time) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             plims_a_hdr_engineering
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943

In [17]:
data_hdreng["preferred_timestamp"][0].values

KeyError: "No variable named 'preferred_timestamp'. Did you mean one of ('sample_timestamp', 'internal_timestamp')?"

In [24]:
# telem_hdreng_path = os.path.abspath(os.path.join("./plims_data", f"dev0_telem_hdr_eng_{datetime.now().isoformat(timespec='seconds')}.nc"))
telem_hdreng_path = os.path.abspath(os.path.join("./plims_data", f"dev0_telem_hdr_eng.nc"))
# telem_hdreng_path = re.sub(":", "", telem_hdreng_path)
data_hdreng.to_netcdf(telem_hdreng_path)

ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (83, 18) and arg 1 with shape (83, 64).

#### Telemetered HDR science data stream

In [ ]:
# Setup parameters needed to request data
refdes = datastreams.refdes[1]
method = datastreams.method[1]
stream = datastreams.stream[1]

In [ ]:
# Import vocabulary
site, node, sensor = refdes.split('-', 2)
vocab = get_vocabulary(site, node, sensor)

In [ ]:
# Use the gold copy THREDDs datasets
thredds_url = M2M.get_thredds_url(refdes, method, stream, beginDT="2024-05-01T00:00:00.0", endDT="2024-05-15T00:00:00.0")
# Get the THREDDs catalog
thredds_catalog = M2M.get_thredds_catalog(thredds_url)
thredds_catalog = [x for x in thredds_catalog if "blank" not in x]

# Clean the THREDDs catalog
sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream)

# Generate the urls to access and load the data
sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["dodsC"], file) for file in sensor_files]

Waiting for request to process
Waiting for request to process
Waiting for request to process


In [ ]:
# Load the data
with ProgressBar():
    data_hdrsci = xr.open_mfdataset(sensor_files, parallel=True)

[########################################] | 100% Completed | 6.13 ss


In [ ]:
# Check dataset content
data_hdrsci

<xarray.Dataset> Size: 30kB
Dimensions:              (obs: 83)
Coordinates:
  * obs                  (obs) int32 332B 0 1 2 3 4 5 6 ... 76 77 78 79 80 81 82
    time                 (obs) datetime64[ns] 664B dask.array<chunksize=(83,), meta=np.ndarray>
Data variables: (12/34)
    pmta_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    rois_per_ml          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    pmtb_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_speed         (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    beads_counter        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    run_fast_factor      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    ...                   ...
    volume_analyzed      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_gap_min         (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_area_min        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_x_grow          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    look_time            (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_num           (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
Attributes: (12/73)
    node:                               RID27
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             plims_a_hdr_instrument
    ...                                 ...
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943
    DODS.strlen:                        18
    DODS.dimName:                       string18
    DODS_EXTRA.Unlimited_Dimension:     obs

In [ ]:
data_hdrsci.to_netcdf(f"./plims_data/dev0_telemetered_hdr_sci_{datetime.now()}.nc")

#### Telemetered ADC science data stream

In [ ]:
# Setup parameters needed to request data
refdes = datastreams.refdes[1]
method = datastreams.method[1]
stream = datastreams.stream[1]

In [ ]:
# Import vocabulary
site, node, sensor = refdes.split('-', 2)
vocab = get_vocabulary(site, node, sensor)

In [ ]:
# Use the gold copy THREDDs datasets
thredds_url = M2M.get_thredds_url(refdes, method, stream, beginDT="2024-05-01T00:00:00.0", endDT="2024-05-15T00:00:00.0")
# Get the THREDDs catalog
thredds_catalog = M2M.get_thredds_catalog(thredds_url)
thredds_catalog = [x for x in thredds_catalog if "blank" not in x]

# Clean the THREDDs catalog
sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream)

# Generate the urls to access and load the data
sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["dodsC"], file) for file in sensor_files]

Waiting for request to process
Waiting for request to process
Waiting for request to process


In [ ]:
# Load the data
with ProgressBar():
    data_adcsci = xr.open_mfdataset(sensor_files, parallel=True)

[########################################] | 100% Completed | 6.13 ss


In [ ]:
# Check dataset content
data_adcsci

<xarray.Dataset> Size: 30kB
Dimensions:              (obs: 83)
Coordinates:
  * obs                  (obs) int32 332B 0 1 2 3 4 5 6 ... 76 77 78 79 80 81 82
    time                 (obs) datetime64[ns] 664B dask.array<chunksize=(83,), meta=np.ndarray>
Data variables: (12/34)
    pmta_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    rois_per_ml          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    pmtb_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_speed         (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    beads_counter        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    run_fast_factor      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    ...                   ...
    volume_analyzed      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_gap_min         (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_area_min        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_x_grow          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    look_time            (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_num           (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
Attributes: (12/73)
    node:                               RID27
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             plims_a_hdr_instrument
    ...                                 ...
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943
    DODS.strlen:                        18
    DODS.dimName:                       string18
    DODS_EXTRA.Unlimited_Dimension:     obs

In [ ]:
data_adcsci.to_netcdf(f"./plims_data/dev0_telemetered_adc_sci_{datetime.now()}.nc")

#### Recovered host HDR engineering data stream

In [ ]:
# Setup parameters needed to request data
refdes = datastreams.refdes[0]
method = datastreams.method[0]
stream = datastreams.stream[0]

In [ ]:
# Import vocabulary
site, node, sensor = refdes.split('-', 2)
vocab = get_vocabulary(site, node, sensor)

In [ ]:
# Use the gold copy THREDDs datasets
thredds_url = M2M.get_thredds_url(refdes, method, stream, beginDT="2024-05-01T00:00:00.0", endDT="2024-05-15T00:00:00.0")
# Get the THREDDs catalog
thredds_catalog = M2M.get_thredds_catalog(thredds_url)
thredds_catalog = [x for x in thredds_catalog if "blank" not in x]

# Clean the THREDDs catalog
sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream)

# Generate the urls to access and load the data
sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["dodsC"], file) for file in sensor_files]



Waiting for request to process
Waiting for request to process
Waiting for request to process


In [ ]:
# Load the data
with ProgressBar():
    data_hdreng = xr.open_mfdataset(sensor_files, parallel=True)

[########################################] | 100% Completed | 7.28 ss


In [ ]:
# Check dataset content
data_hdreng

<xarray.Dataset> Size: 34kB
Dimensions:               (obs: 83)
Coordinates:
  * obs                   (obs) int32 332B 0 1 2 3 4 5 6 ... 77 78 79 80 81 82
    time                  (obs) datetime64[ns] 664B dask.array<chunksize=(83,), meta=np.ndarray>
Data variables: (12/50)
    provenance            (obs) |S64 5kB dask.array<chunksize=(83,), meta=np.ndarray>
    pump1_state           (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    refill_debubble       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    prime_sample          (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    deployment            (obs) int32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    alt_interval          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    ...                    ...
    alt_pmta_threshold    (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    focus_motor_step_lg   (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    syringes_autorun      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    bleach_rinse_ct       (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    auto_start            (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_volume_skip    (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
Attributes: (12/73)
    node:                               RID27
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             plims_a_hdr_engineering
    ...                                 ...
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943
    DODS.strlen:                        18
    DODS.dimName:                       string18
    DODS_EXTRA.Unlimited_Dimension:     obs

In [ ]:
data_hdreng["preferred_timestamp"][0].values

array(b'internal_timestamp', dtype=object)

In [ ]:
data_hdreng.to_netcdf(f"./plims_data/dev0_recoveredhost_hdr_eng_{datetime.now()}.nc")

#### Recovered host HDR science data stream

In [ ]:
# Setup parameters needed to request data
refdes = datastreams.refdes[1]
method = datastreams.method[1]
stream = datastreams.stream[1]

In [ ]:
# Import vocabulary
site, node, sensor = refdes.split('-', 2)
vocab = get_vocabulary(site, node, sensor)

In [ ]:
# Use the gold copy THREDDs datasets
thredds_url = M2M.get_thredds_url(refdes, method, stream, beginDT="2024-05-01T00:00:00.0", endDT="2024-05-15T00:00:00.0")
# Get the THREDDs catalog
thredds_catalog = M2M.get_thredds_catalog(thredds_url)
thredds_catalog = [x for x in thredds_catalog if "blank" not in x]

# Clean the THREDDs catalog
sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream)

# Generate the urls to access and load the data
sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["dodsC"], file) for file in sensor_files]

Waiting for request to process
Waiting for request to process
Waiting for request to process


In [ ]:
# Load the data
with ProgressBar():
    data_hdrsci = xr.open_mfdataset(sensor_files, parallel=True)

[########################################] | 100% Completed | 6.13 ss


In [ ]:
# Check dataset content
data_hdrsci

<xarray.Dataset> Size: 30kB
Dimensions:              (obs: 83)
Coordinates:
  * obs                  (obs) int32 332B 0 1 2 3 4 5 6 ... 76 77 78 79 80 81 82
    time                 (obs) datetime64[ns] 664B dask.array<chunksize=(83,), meta=np.ndarray>
Data variables: (12/34)
    pmta_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    rois_per_ml          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    pmtb_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_speed         (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    beads_counter        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    run_fast_factor      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    ...                   ...
    volume_analyzed      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_gap_min         (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_area_min        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_x_grow          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    look_time            (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_num           (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
Attributes: (12/73)
    node:                               RID27
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             plims_a_hdr_instrument
    ...                                 ...
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943
    DODS.strlen:                        18
    DODS.dimName:                       string18
    DODS_EXTRA.Unlimited_Dimension:     obs

In [ ]:
data_hdrsci.to_netcdf(f"./plims_data/dev0_recoveredhost_hdr_sci_{datetime.now()}.nc")

#### Recovered host HDR science data stream

In [ ]:
# Setup parameters needed to request data
refdes = datastreams.refdes[1]
method = datastreams.method[1]
stream = datastreams.stream[1]

In [ ]:
# Import vocabulary
site, node, sensor = refdes.split('-', 2)
vocab = get_vocabulary(site, node, sensor)

In [ ]:
# Use the gold copy THREDDs datasets
thredds_url = M2M.get_thredds_url(refdes, method, stream, beginDT="2024-05-01T00:00:00.0", endDT="2024-05-15T00:00:00.0")
# Get the THREDDs catalog
thredds_catalog = M2M.get_thredds_catalog(thredds_url)
thredds_catalog = [x for x in thredds_catalog if "blank" not in x]

# Clean the THREDDs catalog
sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream)

# Generate the urls to access and load the data
sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["dodsC"], file) for file in sensor_files]

Waiting for request to process
Waiting for request to process
Waiting for request to process


In [ ]:
# Load the data
with ProgressBar():
    data_adcsci = xr.open_mfdataset(sensor_files, parallel=True)

[########################################] | 100% Completed | 6.13 ss


In [ ]:
# Check dataset content
data_adcsci

<xarray.Dataset> Size: 30kB
Dimensions:              (obs: 83)
Coordinates:
  * obs                  (obs) int32 332B 0 1 2 3 4 5 6 ... 76 77 78 79 80 81 82
    time                 (obs) datetime64[ns] 664B dask.array<chunksize=(83,), meta=np.ndarray>
Data variables: (12/34)
    pmta_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    rois_per_ml          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    pmtb_threshold       (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_speed         (obs) float32 332B dask.array<chunksize=(83,), meta=np.ndarray>
    beads_counter        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    run_fast_factor      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    ...                   ...
    volume_analyzed      (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_gap_min         (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_area_min        (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    blob_x_grow          (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    look_time            (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
    sample_num           (obs) float64 664B dask.array<chunksize=(83,), meta=np.ndarray>
Attributes: (12/73)
    node:                               RID27
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             plims_a_hdr_instrument
    ...                                 ...
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943
    DODS.strlen:                        18
    DODS.dimName:                       string18
    DODS_EXTRA.Unlimited_Dimension:     obs

In [ ]:
data_adcsci.to_netcdf(f"./plims_data/dev0_recoveredhost_adc_sci_{datetime.now()}.nc")

## Check data accuracy

Early versions of the PLIMS NetCDFs showed inaccuracies due to a loss of precision in the float data type (32-bit) for multiple parameters. These were changed to the double data type (64-bit), but the efficacy of the updates need to be confirmed.

In [ ]:
# Define routine to return any offset between the timestamp of record and sample start time
def test_time_delta(data):
    t_delta = data["internal_timestamp"] - data["sample_timestamp"]
    print(t_delta.head)
    check = (t_delta == 0)
    if check.all() == True:
        t_delta = "No offset."
    return t_delta

In [ ]:
# Test for a time delta in both data streams
tdel_hdrsci = test_time_delta(data_hdrsci)
print(tdel_hdrsci)
tdel_hdreng = test_time_delta(data_hdreng)
print(tdel_hdreng)